## Step 1.1 Run the cells before Step 1.2.

In [9]:
!pip install mesh_to_sdf
!apt-get install xvfb
!pip install pyvirtualdisplay

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libfontenc1 libxfont2 libxkbfile1 x11-xkb-utils xfonts-base xfonts-encodings xfonts-utils
  xserver-common
The following NEW packages will be installed:
  libfontenc1 libxfont2 libxkbfile1 x11-xkb-utils xfonts-base xfonts-encodings xfonts-utils
  xserver-common xvfb
0 upgraded, 9 newly installed, 0 to remove and 18 not upgraded.
Need to get 7,815 kB of archives.
After this operation, 11.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfontenc1 amd64 1:1.1.4-1build3 [14.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxfont2 amd64 1:2.0.5-1build1 [94.5 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxkbfile1 amd64 1:1.1.0-1build3 [71.8 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/main amd64 x11-xkb-utils amd64 7.7+5build4 [172 kB]
Get:5 http://archiv

In [ ]:
from IPython import get_ipython
from IPython.display import display

import torch
from torch import nn
from mesh_to_sdf import sample_sdf_near_surface
import trimesh
from torch.utils.data import DataLoader, Dataset
import numpy as np
from math import sqrt
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


## Step 1.2 NeuralSDFDataset

In [ ]:
"""
Step 1.2: Finish the NeuralSDFDataset class that loads a mesh with path `mesh_path` and sample points
"""

class NeuralSDFDataset(Dataset):
    def __init__(self, mesh_path, sample_num, device='cuda'):
        """
        (1) You will first use a package called `trimesh` (it's already imported in Step 1.1) to load a `.obj` file with path <code>mesh_path</code>
        (2) We use sample_num points sampled non uniformly around the surface using non-uniform sampling method `sample_sdf_near_surface`. 
        (3) After sampling points, you will convert sampled points in type `numpy ndarray` to `torch tensor` by using the `torch.from_numpy` function.
            Since we are using Colab where GPU is available, you will put those tensors to CUDA by using `.to(device)`.
        """
        ### you implementation starts
        
        ### you implementation ends


    def __len__(self):
        return 1 # we are not using this

    def __getitem__(self, idx):
        return self.points, self.sdf

In [ ]:
### Helper method for test result of the sampled points from your dataset class.
def test_dataset(sdf_loader_test):
  points, sdf = next(iter(sdf_loader_test))
  points =  points.cpu().detach().numpy().squeeze(0)
  sdf = sdf.cpu().detach().numpy().squeeze()
  norm = plt.Normalize(vmin=np.min(sdf), vmax=np.max(sdf))
  colors = plt.cm.coolwarm(norm(sdf))
  fig = plt.figure(figsize=(8, 6))
  ax = fig.add_subplot(111, projection='3d')

  sc = ax.scatter(points[:, 0], points[:, 1], points[:, 2], c=sdf, cmap='coolwarm', marker='o')

  cbar = plt.colorbar(sc, ax=ax, shrink=0.5)
  cbar.set_label("SDF Value")

  ax.set_xlabel("X")
  ax.set_ylabel("Y")
  ax.set_zlabel("Z")
  ax.set_title("3D Point Cloud Visualization with SDF Values")
  ax.view_init(elev=0, azim=0)
  plt.show()

  
sample_num = 10000
device='cuda'
mesh_path="cow.obj" ### Change to bunny.obj if needed.

sdf_test = NeuralSDFDataset(mesh_path, sample_num, device=device)
sdf_loader_test = DataLoader(sdf_test, num_workers=0)
test_dataset(sdf_loader_test)

## Step 1.3 Network Structure

In [ ]:
"""
Sine layer
"""

class SineLayer(nn.Module):
    """
    Default sin activation frequency w0 is set to be 30, feel free to play with it.
    However, we set this to be 15 by default due to our network is much smaller that suffers from learning high frequency features.
    If you have time, make the hidden layers to 512 width with 5 depth, then checkout the difference.

    By default, first layer's weight is initialized differently as suggested in Sec.3.2 in the original paper. We use is_first flag to
    check whether we should init the weights differently.

    We use linear layer as the last layer without any activation functions since SDF values shouldn't be limited to a certain range.
    We use is_last flag to check if we should use activation functions or not.
    """

    def __init__(self, in_features, out_features, bias=True,
                 is_first=False, is_last=False, w0=15, skip_weight=1):
        super().__init__()
        self.w0 = w0
        self.is_first = is_first
        self.is_last = is_last
        self.skip_weight = skip_weight

        self.in_features = in_features

        ### your implementation starts

        ### your implementation ends

        self.init_weights()

    def init_weights(self):
        """
        initialize the weights for first layer and other layers following last paragraph in Sec.3.2
        """
        with torch.no_grad():
            if self.is_first:
                self.fc.weight.uniform_(-1. / self.in_features,
                                             1. / self.in_features)
            else:
                self.fc.weight.uniform_(-np.sqrt(6 / self.in_features) / self.w0,
                                             np.sqrt(6 / self.in_features) / self.w0)

    def forward(self, x):
        """
        Use sin activation function for fully connected layers, notice we add a skip connection to make network learns the shape easier since
        we don't have enough neurons.
        """
        ### your implementation starts

        ### your implementation ends

"""
Step 1.3: Network structure
"""
class NeuralSDF(nn.Module):
    def __init__(self, in_features, hidden_features, hidden_layers, out_features, w0=30):
        super().__init__()

        """
        Weights in first layer should be initialized differently from other layers (See last sentence in Sec.3.2 in SIREN paper)
        Notice we don't use activation for the final layer.
        """
        self.net = []
        self.w0 = w0
        self.hidden_features = hidden_features
        self.hidden_layers = hidden_layers
        self.out_features = out_features
        self.in_features = in_features

        ### your implementation starts

        ### your implementation ends
        self.net = nn.Sequential(*self.net)

    def forward(self, x):
        output = self.net(x)
        return output

## Step 1.4 Train Your Network

In [ ]:
"""
We train neuralSDF for 20000 epochs, feel free to train more if time allowed.
`lr` is set to 1e-4, feel free to play around with this.
"""
def train_neuralSDF(dataloader, hidden_features, hidden_layers, w0, lr=1e-4, iterations=10000, device='cuda'):
    model = NeuralSDF(in_features=3, out_features=1, hidden_features=hidden_features, hidden_layers=hidden_layers, w0=w0).to(device)
    optimizer = torch.optim.Adam(lr=lr, params=model.parameters(), weight_decay=.0)
    data, labels = next(iter(dataloader))

    for epoch in range(iterations):

        ### your implementation starts

        ### your implementation ends
        if epoch % 500 == 0:
            print(f'Epoch {epoch+1}, Loss: {loss.item()}')

    return model

In [ ]:
"""
sample_num: total points sampled (feel free to increase this if needed)
mesh_path: relative path to .obj file location
"""

sample_num = 300000  ### total number of points sampled as training points, feel free to change this.
device='cuda'
mesh_path="cow.obj" ### mesh path to your mesh,

sdf = NeuralSDFDataset(mesh_path, sample_num, device=device)
sdfloader = DataLoader(sdf, num_workers=0)

In [ ]:
"""
hidden_features: hidden layer width.
hidden_layers: hidden layer depth.
w0: Activation frequency. We suggest 15 for our given examples.

Feel free to play around with these parameters.
"""
hidden_features = 16 ### hidden layer width, feel free to change
hidden_layers = 2 ### hidden layer depth, feel free to change
w0 = 15 ### activation function frequency, feel free to change
iterations = 10000 ### total number of training iterations, feel free to change
lr = 1e-4 ### learning rate, feel free to change

neural_sdf = train_neuralSDF(sdfloader, hidden_features = hidden_features, hidden_layers = hidden_layers, w0 = w0, lr=lr, iterations=iterations, device=device)

Epoch 1, Loss: 0.033336713910102844
Epoch 501, Loss: 0.000399589043809101
Epoch 1001, Loss: 0.00013242478598840535
Epoch 1501, Loss: 7.75775988586247e-05
Epoch 2001, Loss: 5.152824451215565e-05
Epoch 2501, Loss: 3.68651635653805e-05
Epoch 3001, Loss: 2.8019421733915806e-05
Epoch 3501, Loss: 2.2144828108139336e-05
Epoch 4001, Loss: 1.809777313610539e-05
Epoch 4501, Loss: 1.5428755432367325e-05
Epoch 5001, Loss: 1.3461575690598693e-05
Epoch 5501, Loss: 1.1997304682154208e-05
Epoch 6001, Loss: 1.0885492883971892e-05
Epoch 6501, Loss: 1.0055349775939249e-05
Epoch 7001, Loss: 9.42936912906589e-06
Epoch 7501, Loss: 8.932043783715926e-06
Epoch 8001, Loss: 8.521431482222397e-06
Epoch 8501, Loss: 8.263333256763872e-06
Epoch 9001, Loss: 8.136436008499004e-06
Epoch 9501, Loss: 7.687694960623048e-06


## Step 2 Copy Network Weights to Shader 

In [ ]:

"""
Modified based on https://www.shadertoy.com/view/wtVyWK
Will be used for outputing network weights to matrices and visualize in our shader
"""
import re

### Helper function for convert pytorch cuda tensor to numpy arrays
def dump_data(dat):
  dat = dat.cpu().detach().numpy()
  return dat

### Print a vector to a form that's usable in fragement shader
def print_vec4(ws):
  vec = "vec4(" + ",".join(["{0:.2f}".format(w) for w in ws]) + ")"
  vec = re.sub(r"\b0\.", ".", vec)
  return vec

### Print a matrix to a form that's usable in fragement shader
def print_mat4(ws):
  mat = "mat4(" + ",".join(["{0:.2f}".format(w) for w in np.transpose(ws).flatten()]) + ")"
  mat = re.sub(r"\b0\.", ".", mat)
  return mat

### Since we know networks are just matrices and vectors, this function converts our network to matrices and vectors that 
### can be compiled in fragement shader. 
def serialize_to_shadertoy(network, varname):
  omega = network.w0
  chunks = int(network.hidden_features/4)
  lin = network.net[0].fc
  in_w = dump_data(lin.weight)
  in_bias = dump_data(lin.bias)
  om = omega
  for row in range(chunks):
    line = "vec4 %s0_%d=sin(" % (varname, row)
    for ft in range(network.in_features):
        feature = x_vec = in_w[row*4:(row+1)*4,ft]*om
        line += ("p.%s*" % ["y","z","x"][ft]) + print_vec4(feature) + "+"
    bias = in_bias[row*4:(row+1)*4]*om
    line += print_vec4(bias) + ");"
    print(line)

  #hidden layers
  for layer in range(network.hidden_layers):
    layer_w = dump_data(network.net[layer+1].fc.weight)
    layer_bias = dump_data(network.net[layer+1].fc.bias)
    for row in range(chunks):
      line = ("vec4 %s%d_%d" % (varname, layer+1, row)) + "=sin("
      for col in range(chunks):
        mat = layer_w[row*4:(row+1)*4,col*4:(col+1)*4]*omega
        line += print_mat4(mat) + ("*%s%d_%d"%(varname, layer, col)) + "+\n    "
      bias = layer_bias[row*4:(row+1)*4]*omega
      line += print_vec4(bias)+")/%0.1f+%s%d_%d;"%(sqrt(layer+1), varname, layer, row)
      print(line)

  #output layer
  out_w = dump_data(network.net[-1].fc.weight)
  out_bias = dump_data(network.net[-1].fc.bias)
  for outf in range(network.out_features):
    line = "return "
    for row in range(chunks):
      vec = out_w[outf,row*4:(row+1)*4]
      line += ("dot(%s%d_%d,"%(varname, network.hidden_layers, row)) + print_vec4(vec) + ")+\n    "
    print(line + "{:0.3f}".format(out_bias[outf])+";")

In [ ]:
serialize_to_shadertoy(neural_sdf, 'f')

vec4 f0_0=sin(p.y*vec4(-2.27,-.42,3.11,1.32)+p.z*vec4(4.14,-.04,2.50,-4.02)+p.x*vec4(2.39,3.72,-4.08,-1.55)+vec4(-2.69,5.49,8.15,5.66));
vec4 f0_1=sin(p.y*vec4(1.94,-3.90,1.04,-3.08)+p.z*vec4(-1.11,1.31,3.88,-.57)+p.x*vec4(-3.48,-.79,3.44,-1.87)+vec4(8.85,1.33,3.54,-4.91));
vec4 f0_2=sin(p.y*vec4(-.50,-.01,-2.68,-.59)+p.z*vec4(2.72,1.88,-.77,-3.06)+p.x*vec4(1.83,-.55,.87,-3.82)+vec4(-1.80,2.33,7.17,-2.31));
vec4 f0_3=sin(p.y*vec4(2.75,1.26,-2.49,-1.01)+p.z*vec4(-2.49,2.80,3.07,-.10)+p.x*vec4(-.73,-3.07,-2.46,1.27)+vec4(-4.81,-6.38,6.50,1.89));
vec4 f1_0=sin(mat4(.12,.20,-.25,-.25,.53,.09,-.55,-.15,-.55,.66,.15,.32,-.29,.04,-.12,-.27)*f0_0+
    mat4(-.04,.22,-.01,-.70,-.43,-.02,.41,.76,-.08,-.03,-.14,.54,-.10,.74,.15,-.07)*f0_1+
    mat4(.48,.38,-.31,.43,.51,-.11,.85,-.07,.46,-.35,.96,-.49,-.64,-.43,.21,.27)*f0_2+
    mat4(-.14,-.17,.02,.27,.12,.33,.03,-.25,-.28,-.15,.16,-.36,.60,.74,.48,-.09)*f0_3+
    vec4(-.22,-.57,-3.53,-.77))/1.0+f0_0;
vec4 f1_1=sin(mat4(-.23,-.29,.71,-.49,.39,.20,